#### Dependencies

In [19]:
# !pip install pandas
# !pip install numpy
# !pip install opencv-python
# !pip install pytesseract
# !dnf install tesseract-ocr
# !dnf install poppler-utils
# !pip install -U spacy
# !python -m spacy download en_core_web_sm
# !pip install fitz
# !pip install PyPDF2 pdf2image


# Creating a model that takes in the images and processes them


## Image Processing

### Collect all PDF's and change them to image format


In [20]:
import os
import glob
import PyPDF2
from pdf2image import convert_from_path

# Path to PDFs
directory_path = './Prod files/'

# Extension to be captured
pdf_extension = '*.pdf'

# Create a directory to save the extracted images
output_directory = './extracted_images/'
os.makedirs(output_directory, exist_ok=True)

def extract_images_from_pdf(pdf_path, output_dir):
    try:
        # Open the PDF file using PyPDF2
        with open(pdf_path, 'rb') as pdf_file:
            full_name = os.path.basename(pdf_path)
            file_name = os.path.splitext(full_name)
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            num_pages = len(pdf_reader.pages)

            # Convert each page of the PDF to images using pdf2image
            images = convert_from_path(pdf_path)

            for page_num, image in enumerate(images, start=1):
                # Save the image as PNG
                image_filename = os.path.join(output_dir, f'{file_name[0]}page{page_num}.png')
                image.save(image_filename, 'PNG')

    except Exception as e:
        print(f"Error extracting images from {pdf_path}: {str(e)}")

# Find PDF files in the directory
pdf_files = glob.glob(os.path.join(directory_path, pdf_extension))

# Loop through each PDF and extract images
for pdf_path in pdf_files:
    extract_images_from_pdf(pdf_path, output_directory)

print("Image extraction completed.")


Image extraction completed.


## extract the text from the images

In [21]:
import pytesseract
import cv2
import os
import glob

# Define the directory path where your invoice images are located
directory_path = "./extracted_images/"

# Extensions to be captured
image_extensions = ['*.jpg', '*.png']

# List to store the captured data
image_data = []

i = 0

# Tesseract configuration options (you can customize these)
custom_config = r'--oem 3 --psm 6'

for extension in image_extensions:
    pattern = os.path.join(directory_path, extension)
    image_files = glob.glob(pattern)
    for img_path in image_files:
        full_name = os.path.basename(img_path)
        file_name = os.path.splitext(full_name)

        # Load image
        img = cv2.imread(img_path)

        # Convert image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply threshold to convert to binary image
        threshold_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

        # Additional pre-processing steps can be added here (e.g., resizing, denoising)

        # Pass the pre-processed image through pytesseract with custom config
        text = pytesseract.image_to_string(threshold_img, config=custom_config, lang='eng')

        name = file_name[0]

        # Print the extracted text
        image_data.append([name, text])
        i += 1
        print('Image', i)


Image 1
Image 2
Image 3
Image 4
Image 5
Image 6
Image 7
Image 8
Image 9
Image 10


## Save the extracted text in a df 

In [22]:
import pandas as pd
text = pd.DataFrame(image_data,columns=("image_name","text"))
text.to_csv("./Prod files/extracted_text.csv")

## Begin the Training process